In [80]:
#작업에 필요한 라이브러리 
from selenium import webdriver 
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By

#01. 웹 열기
dr = webdriver.Chrome("C:/windows/chromedriver.exe") #웹드라이버로 크롬 웹 켜기
dr.implicitly_wait(3)
dr.set_window_size(414, 800) 	#브라우저 크기 414*800으로 고정
dr.get('https://www.instagram.com/') #인스타그램 웹 켜기
time.sleep(2) 	#2초 대기

C:\Users\admin\AppData\Local\Temp\ipykernel_38812\3946941392.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  dr = webdriver.Chrome("C:/windows/chromedriver.exe") #웹드라이버로 크롬 웹 켜기


In [81]:
# 02. 로그인
#경로 지정

id_box = dr.find_element(By.CSS_SELECTOR,"#loginForm > div > div:nth-child(1) > div > label > input")   #아이디 입력창
password_box = dr.find_element(By.CSS_SELECTOR,"#loginForm > div > div:nth-child(2) > div > label > input")     #비밀번호 입력창
login_button = dr.find_element(By.CSS_SELECTOR,'#loginForm > div > div:nth-child(3) > button')      #로그인 버튼

#동작 제어
act = ActionChains(dr)      #동작 명령어 지정
act.send_keys_to_element(id_box, 'iambowonkang').send_keys_to_element(password_box, 'bo5nicd').click(login_button).perform()     #아이디 입력, 비밀 번호 입력, 로그인 버튼 클릭 수행
time.sleep(2)

In [67]:
# 필요 패키지 호출
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time

In [68]:
# 함수 정의: 검색어 조건에 따른 url 생성
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

In [69]:
# 함수 정의: 열린 페이지에서 첫 번째 게시물 클릭 + sleep 메소드 통하여 시차 두기
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR,"div._aagw")[0]
    first.click()
    time.sleep(3)

In [70]:
# 함수 정의: 본문 내용, 작성일자, 좋아요 수, 위치 정보, 해시태그 가져오기
import re
from bs4 import BeautifulSoup

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
    except:
        content = ''
    # 해시태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 작성일자
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    # 좋아요
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade')[0].findAll('span')[-1].text
    except:
        like = 0
    # 위치
    try:
        place = soup.select('div._aaqm')[0].text
    except:
        place = ''

    data = [content, date, like, place, tags]
    return data

In [71]:
# 함수 정의: 첫 번째 게시물 클릭 후 다음 게시물 클릭
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR,"div._aaqg._aaqh") # 2022/01/11 수정
    right.click()
    time.sleep(3)

In [82]:
# 게시물을 조회할 검색 키워드 입력 요청
word = input("검색어를 입력하세요 : ")
word = str(word)
url = insta_searching(word)

In [83]:
# 검색 결과 페이지 열기
dr.get(url)
time.sleep(10) # 코드 수행 환경에 따라 페이지가 로드되는 데 시간이 더 걸릴 수 있어 8초로 변경(2022/01/11)

# 첫 번째 게시물 클릭
select_first(dr)

# 본격적으로 데이터 수집 시작
results = []
## 수집할 게시물의 수
target = 10
for i in range(target):

    try:
        data = get_content(dr)
        results.append(data)
        move_next(dr)
    except:
        time.sleep(2)
        move_next(dr)
    time.sleep(5)

print(results[:2])

[['(광고) @Stay_hwaga #busantravel #busanhotel기장에 새로 오픈한 오션뷰 펜션, 스테이화가를 소개합니다!마치 화가의 집에 하루 머물며휴식을 즐길 수 있는 컨셉을 가진기장 new 펜션 스테이 화가 🎨특색 있는 컨셉에 맞게기장 바다를 바라보며 반신욕을 즐기고,우리만의 추억이 담긴 그림까지 그릴 수 있다고 하는 데요아이보리 빛의 인테리어와 단란한 소품들로 꾸며져독특하지만 감성적이고 수더분한 분위기로공간 자체만으로 힐링이 되는 스테이랍니다:)✨부산에서의 하루를기장 스테이화가에서 만끽해보시는 건 어떠세요?📍기장군 기장읍 연화리 326-2 스테이화가📍객실 이용 전고객 조식바구니 제공📍나만의 명화 채색도구📍폴라로이드 포토 프린터#busanbnb #gijang', '2023-01-12', 0, '', ['#busantravel', '#busanhotel기장에', '#busanbnb', '#gijang']], ['그런 찰나의 기억속에잠시 모든걸 내려놓고쉴 수 있다면.....🌃', '2023-01-11', '815', '부산광역시', []]]


In [84]:
# 결과를 데이터프레임으로 저장
import pandas as pd
from datetime import datetime

date = datetime.today().strftime('%Y-%m-%d')

results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like','place','tags']
results_df.to_excel(date + '_about '+word+' insta crawling.xlsx')

In [ ]:
# #03. 프로필 들어가기
# #프로필 사진 클릭하기
# my_photo = dr.find_element(By.CSS_SELECTOR,'#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.ctQZg.KtFt3 > div > div:nth-child(6) > div.EforU > span > img')
# my_photo.click()
# time.sleep(2)

# #프로필 버튼 클릭하기
# profile = dr.find_element(By.CSS_SELECTOR,'section > nav > div._8MQSO.Cx7Bp > div > div > div.ctQZg > div > div:nth-child(6) > div.poA5q > div.uo5MA._2ciX.tWgj8.XWrBI > div._01UL2 > a:nth-child(1) > div')
# profile.click()
# time.sleep(2)